Resources about numpy:
https://numpy.org/doc/stable/user/quickstart.html

Resources about pandas:
https://pandas.pydata.org/docs/reference/frame.html


In [20]:
!pip install numpy

In [21]:
!pip3 install pandas

In [22]:
!pip install openpyxl

In [23]:
# Import numpy and pandas
import numpy as np
import pandas as pd

In [24]:
# TODO: Create function of code below called system_reader which takes the file path of the system data as input
# and returns the line and bus data.

In [25]:
# Replace 'file.xlsx' with your actual file path
file_path = 'system_basecase.xlsx'

# Replace 'Sheet1' with the name of the sheet you want to read
sheet_name1 = 'BusData'
sheet_name2 = 'LineData'


# Read the Excel file into a pandas DataFrame for the specified sheet
bus_data = pd.read_excel(file_path, sheet_name=sheet_name1)
line_data = pd.read_excel(file_path, sheet_name=sheet_name2)

# Display the contents of the DataFrame
print(bus_data)
print(line_data)

    Bus #  P MW  Q MVAr Type  P Gen  V Set
0       1   0.0     0.0    S      0  1.040
1       2  20.1    12.5    D      0  1.000
2       3  45.2    16.0    G     42  1.025
3       4  60.8    10.9    D      0  1.000
4       5   7.6    -1.6    D      0  1.000
5       6  13.5     8.5    D      0  1.000
6       7  33.5    11.6    D      0  1.000
7       8   9.0     5.8    D      0  1.000
8       9   4.3     2.1    G     23  1.020
9      10   5.2     1.6    G     27  1.020
10     11  13.5     5.8    D      0  1.000
11     12  14.9     5.0    G     33  1.030
    From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0      1   2       0.01938       0.05917        0.0528         95
1      1   5       0.05203       0.20304        0.0492        100
2      2   3       0.04699       0.19797        0.0438       9999
3      2   4       0.05811       0.17632        0.0340       9999
4      2   5       0.05695       0.07388        0.0346       9999
5      3   4       0.67010       0.17103     

In [26]:
# TODO: Create function of the code below called admittance_matrix

In [27]:
# Calculating admittance matrix
num_buses = len(bus_data)
Ybus = np.zeros((num_buses, num_buses), dtype=complex)

# Test admitance matrix
# print(Ybus)

# Calculate total admittance of lines connected to each bus and store on the main diagonal
for i in range(num_buses):
    # Debug code
    # print('iteration', i)
    
    # Select all lines connected to the current bus in question.
    bus_lines = line_data[(line_data['From'] == i + 1) | (line_data['To'] == i + 1)]
    print(bus_lines)
    # Sum all line admittances.
    total_admittance = sum(1 / (line['Rtotal, p.u.'] + 1j * line['Xtotal, p.u.']) for _, line in bus_lines.iterrows())
    Ybus[i, i] = total_admittance

# Test calculations
# print(Ybus)

# Calculate admittance between each pair of buses and fill the off-diagonal elements
for _, line in line_data.iterrows():
    from_bus = int(line['From']) - 1
    to_bus = int(line['To']) - 1
    
    admittance = 1 / (line['Rtotal, p.u.'] + 1j * line['Xtotal, p.u.'])
    
    Ybus[from_bus, to_bus] = -admittance
    Ybus[to_bus, from_bus] = -admittance  # Assuming symmetric matrix

# print("Admittance Matrix (Ybus):")
print(Ybus)

   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0     1   2       0.01938       0.05917        0.0528         95
1     1   5       0.05203       0.20304        0.0492        100
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0     1   2       0.01938       0.05917        0.0528         95
2     2   3       0.04699       0.19797        0.0438       9999
3     2   4       0.05811       0.17632        0.0340       9999
4     2   5       0.05695       0.07388        0.0346       9999
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
2     2   3       0.04699       0.19797        0.0438       9999
5     3   4       0.67010       0.17103        0.0128       9999
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
3     2   4       0.05811       0.17632        0.0340       9999
5     3   4       0.67010       0.17103        0.0128       9999
6     4   5       0.01335       0.04211        0.0391       9999
7     4   7       0.00000

In [28]:
# Create 12x1 Matrix to store given V values
num_buses = len(bus_data)
V_bus = np.zeros(num_buses)
# print(num_buses)
for i in range(len(bus_data)):
    V_bus[i] = bus_data.iloc[i]['V Set']
# print(V_bus)

In [29]:
# Create 12 x 1 Matrix to initialize bus angle values
bus_angles = np.zeros(len(bus_data))
# print(bus_angles)

In [30]:
base_MVA = 100  # Base MVA

# Create a matrix to store per-unit values for active power (P), reactive power (Q), and generated power (P Gen)
num_buses = len(bus_data)
per_unit_matrix = np.zeros((num_buses, 3))  # 3 columns for P, Q, and P Gen

# Calculate and store per-unit values in the matrix
for i in range(num_buses):
    P_pu = bus_data.iloc[i]['P MW'] / base_MVA
    Q_pu = bus_data.iloc[i]['Q MVAr'] / base_MVA
    P_Gen_pu = bus_data.iloc[i]['P Gen'] / base_MVA
    
    # Store per-unit values in the matrix
    per_unit_matrix[i, 0] = P_pu
    per_unit_matrix[i, 1] = Q_pu
    per_unit_matrix[i, 2] = P_Gen_pu

# Display the matrix
print("Per-Unit Matrix (Active Power (P), Reactive Power (Q), Generated Power (P Gen)):")
print(per_unit_matrix)

Per-Unit Matrix (Active Power (P), Reactive Power (Q), Generated Power (P Gen)):
[[ 0.     0.     0.   ]
 [ 0.201  0.125  0.   ]
 [ 0.452  0.16   0.42 ]
 [ 0.608  0.109  0.   ]
 [ 0.076 -0.016  0.   ]
 [ 0.135  0.085  0.   ]
 [ 0.335  0.116  0.   ]
 [ 0.09   0.058  0.   ]
 [ 0.043  0.021  0.23 ]
 [ 0.052  0.016  0.27 ]
 [ 0.135  0.058  0.   ]
 [ 0.149  0.05   0.33 ]]


In [31]:
#Calculating the net injection

df = pd.DataFrame(per_unit_matrix, columns=['P, p.u.', 'Q, p.u.', 'P_Gen, p.u.'])

# Perform column operations
netInjection_matrix = df.copy()
netInjection_matrix['P, p.u.'] = netInjection_matrix['P_Gen, p.u.'] - netInjection_matrix['P, p.u.']
netInjection_matrix['Q, p.u.'] = -1 * netInjection_matrix['Q, p.u.']

# Rename columns in the new matrix
netInjection_matrix.columns = ['P_net, p.u.', 'Q_net, p.u.', 'P_Gen, p.u.']

print(netInjection_matrix)


    P_net, p.u.  Q_net, p.u.  P_Gen, p.u.
0         0.000       -0.000         0.00
1        -0.201       -0.125         0.00
2        -0.032       -0.160         0.42
3        -0.608       -0.109         0.00
4        -0.076        0.016         0.00
5        -0.135       -0.085         0.00
6        -0.335       -0.116         0.00
7        -0.090       -0.058         0.00
8         0.187       -0.021         0.23
9         0.218       -0.016         0.27
10       -0.135       -0.058         0.00
11        0.181       -0.050         0.33


In [32]:
# Assuming Ybus is the calculated admittance matrix and bus_data contains bus information

# x @ y for matric multiplication

# Number of buses
num_buses = len(bus_data)

# Initialize arrays to store calculated P and Q
P_calc = np.zeros(num_buses)
Q_calc = np.zeros(num_buses)
theta = 0
# Calculate P and Q injections at each bus with zero angles assumption
    
for i in range(num_buses):
    # Compute P and Q injections based on the admittance matrix and assuming zero angles
    P_calc[i] = V_bus[i] * (np.sum(V_bus[j] * (np.real(Ybus[i, j])) * np.cos(theta) \
                                       + np.imag(Ybus[i,j]) * np.sin(theta) for j in range(num_buses)))
    
    Q_calc[i] = V_bus[i] * (np.sum(V_bus[j] * (np.real(Ybus[i, j])) * np.sin(theta) \
                                       - np.imag(Ybus[i,j]) * np.cos(theta) for j in range(num_buses)))

#print('P_calc' , P_calc)
#print('Q_calc' , Q_calc)

# Calculate mismatches  dP = P_k - P_net
mismatch_P = netInjection_matrix['P_net, p.u.'] - P_calc

# Calculate mismatches  dQ = Q_k - Q_net
mismatch_Q = netInjection_matrix['Q_net, p.u.'] - Q_calc



# Display mismatches
print("Mismatch in Active Power (P):")
print(mismatch_P)

print("\nMismatch in Reactive Power (Q):")
print(mismatch_Q)


# for P, eliminate Slack busses
# for Q, eleminate all but gen buses 


Mismatch in Active Power (P):
0    -0.257232
1     0.027341
2    -0.096987
3    -0.572974
4    -0.028627
5    -0.065380
6    -0.292280
7    -0.052382
8     0.108747
9     0.136094
10   -0.051110
11    0.101865
Name: P_net, p.u., dtype: float64

Mismatch in Reactive Power (Q):
0    -9.237056e-16
1    -1.250000e-01
2    -1.600000e-01
3    -1.090000e-01
4     1.600000e-02
5    -8.500000e-02
6    -1.160000e-01
7    -5.800000e-02
8    -2.100000e-02
9    -1.600000e-02
10   -5.800000e-02
11   -5.000000e-02
Name: Q_net, p.u., dtype: float64


/var/folders/3h/60tkw2nd3cl5jw41dhky8jcc0000gn/T/ipykernel_73656/2189205079.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  P_calc[i] = V_bus[i] * (np.sum(V_bus[j] * (np.real(Ybus[i, j])) * np.cos(theta) \
/var/folders/3h/60tkw2nd3cl5jw41dhky8jcc0000gn/T/ipykernel_73656/2189205079.py:19: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  Q_calc[i] = V_bus[i] * (np.sum(V_bus[j] * (np.real(Ybus[i, j])) * np.sin(theta) \


In [33]:
'''
Z=1/y. 
if we delete one of the lines, 1/2 R. 
Cont. plan1 --
'''

'\nZ=1/y. \nif we delete one of the lines, 1/2 R. \nCont. plan1 --\n'